## TEST of SHiP-Detection func

In [ ]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")
import os
import ALL_IN_ONE.Pythia8.product_test as pt
import ALL_IN_ONE.Pythia8.functions_for_run as fr
from numba import jit
import ALL_IN_ONE.Pythia8.cross_section as cs
from tqdm import tqdm
import ALL_IN_ONE.Pythia8.run_save as rs

@jit(nopython=True)

def add_whether_in_the_detector_without_Decay_calcu_add_cross_section_SHiP(filename, out_folder_path):
    rs.mkdir_1(out_folder_path)
    file_path_only, file_name_only = os.path.split(filename)
    file_parent_path_only = os.path.dirname(file_path_only)
    llp_data = pd.read_csv(filename)
    llp_whether_in_detector_SHiP = llp_data.apply(
        lambda row: pt.is_point_inside_frustum_numba_array([row['decay_pos_x'], row['decay_pos_y'], row['decay_pos_z']]),
        axis=1
    )


    cross_section = cs.calculate_cross_section(llp_data)

    llp_data['detected_SHiP'] = llp_whether_in_detector_SHiP
    llp_data['cross_section'] = cross_section
    llp_data['detector_acceptance_SHiP'] = sum(llp_data['detected_SHiP']) / cs.counting_total_LLP(llp_data)
    final_data_folder = file_parent_path_only + '/Completed_llp_data_precise_cross_section'
    fr.mkdir_1(final_data_folder)
    final_data_path = os.path.join(final_data_folder + f'/final_data_cross_section_{file_name_only}')
    llp_data.to_csv(final_data_path, index = False)
    return final_data_folder

def SHiP(LLP_data_folder_dir):
    for files in tqdm(os.listdir(LLP_data_folder_dir)):
        file_path_all = os.path.join(LLP_data_folder_dir, files)
        if os.path.isfile(file_path_all):
            try:
                completed_data_folder = add_whether_in_the_detector_without_Decay_calcu_add_cross_section_SHiP(file_path_all, LLP_data_folder_dir)
            except Exception as e:
                print(f"Error with file: {file_path_all}")
                print(f"Error message: {str(e)}")
                continue
                file_path_all = ''

    return 'Detection and Calcu Cross-Section Completed', completed_data_folder



处理 320000 个点耗时: 0.5645 秒
平均每个点耗时: 1.7640 微秒
内部点的数量: 0


In [2]:
folder = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/SHiP'
SHiP(folder)

100%|██████████| 1/1 [00:00<00:00, 125.43it/s]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/SHiP/test.csv
Error message: Failed in nopython mode pipeline (step: nopython frontend)
NameError: name 'mkdir_1' is not defined
During: Pass nopython_type_inference


UnboundLocalError: local variable 'completed_data_folder' referenced before assignment